In [1]:
import pyautogui
import time
import keyboard
import os
import random
import pygetwindow as gw
import numpy as np
from enum import Enum
from ReadWriteMemory import ReadWriteMemory

# Controller Setup

In [2]:
def goRight():
    pyautogui.keyDown('right')
    time.sleep(2)
    pyautogui.keyUp('right')

def goLeft():
    pyautogui.keyDown('left')
    time.sleep(2)
    pyautogui.keyUp('left')

def jump():
    pyautogui.keyDown('space')
    time.sleep(0.5)
    pyautogui.keyUp('space')

def quickJump():
    pyautogui.keyDown('space')
    time.sleep(0.05)
    pyautogui.keyUp('space')

def crouch():
    pyautogui.keyDown('down')
    time.sleep(0.2)
    pyautogui.keyUp('down')

def long_crouch():
    pyautogui.keyDown('down')
    time.sleep(1)
    pyautogui.keyUp('down')

def doubleJump():
    quickJump()
    quickJump()
    
def quickRight():
    pyautogui.keyDown('right')
    time.sleep(0.2)
    pyautogui.keyUp('right')

def quickLeft():
    pyautogui.keyDown('left')
    time.sleep(0.2)
    pyautogui.keyUp('left')
    
def climb():
    pyautogui.keyDown('up')
    time.sleep(2)
    pyautogui.keyUp('up')

def roll():
    pyautogui.keyDown('ctrl')
    time.sleep(0.2)
    pyautogui.keyUp('ctrl')

def air_roll():
    doubleJump()
    roll()
    
def weapon1():
    pyautogui.keyDown('1')
    time.sleep(0.1)
    pyautogui.keyUp('1')

def weapon2():
    pyautogui.keyDown('2')
    time.sleep(0.5)
    pyautogui.keyUp('2')

def attack_combo1():
    weapon1()
    weapon1()
    weapon1()

def attack_combo():
    weapon1()
    weapon1()
    roll()
    weapon1()
    weapon1()

def health():
    pyautogui.keyDown('h')
    time.sleep(0.5)
    pyautogui.keyUp('h')

def collect():
    pyautogui.keyDown('r')
    time.sleep(0.5)
    pyautogui.keyUp('r')
    
def hardStomp():
    doubleJump()
    pyautogui.keyDown('down')
    quickJump()
    time.sleep(0.1)
    pyautogui.keyUp('down')

def jumpDown():
    pyautogui.keyDown('down')
    quickJump()
    time.sleep(0.1)
    pyautogui.keyUp('down')

#Restart Level
def restart():
    pyautogui.keyDown('esc')
    time.sleep(0.1)
    pyautogui.keyDown('right')
    time.sleep(0.1)
    pyautogui.keyUp('right')
    time.sleep(0.5)
    pyautogui.keyDown('right')
    time.sleep(0.1)
    pyautogui.keyUp('right')
    pyautogui.keyDown('enter')
    time.sleep(0.5)
    pyautogui.keyDown('space')

In [3]:
#define action list
actionList = [goRight, goLeft, jump, 
              quickJump, crouch, long_crouch, 
              doubleJump, quickRight, quickLeft,
              climb, roll, weapon1,
              weapon2, attack_combo,
              collect, hardStomp, jumpDown]

In [4]:
from enum import Enum

class Gamestate(Enum):
    Enemy_fight = 1
    Enemy_escape = 2
    Explore = 3
    Stuck = 4
    Obstacle_Door = 5
    Obstacle_Wall = 6
    Obstacle_Floor = 7

# Action Space

In [5]:
# Define gamestates and actions
gamestates = [Gamestate.Enemy_fight, Gamestate.Enemy_escape, Gamestate.Explore, 
              Gamestate.Stuck, Gamestate.Obstacle_Door, Gamestate.Obstacle_Wall, Gamestate.Obstacle_Floor]

In [6]:
import random

def take_action(state):
    
    actions = list(action_space[state].keys())
    
    probabilities = list(action_space[state].values())
    
    return random.choices(actions, weights=probabilities)[0]

In [7]:
def generate_random_weights(mean, std_dev, num_actions):
    return list(np.random.normal(mean, std_dev, num_actions))

In [8]:
def generate_random_action_space(gamestates, actions):
    action_space = {}
    mean = 5
    std_dev = 1
    num_actions = len(actions)
    
    for gamestate in gamestates:
        action_probabilities = {}
        random_weights = generate_random_weights(mean, std_dev, num_actions)
        index = 0
        for action in actions:
            probability = random_weights[index]
            action_probabilities[action] = probability
            index += 1                
        action_space[gamestate] = action_probabilities
    
    return action_space

In [9]:
def print_action_space(random_action_space):
    # Print the generated random action space
    for gamestate, action_probabilities in random_action_space.items():
        print(f"{gamestate}: {action_probabilities}")

In [10]:
def reinforce_action(action_space, mean, factor):
    updated_action_space = {}

    for gamestate, action_probabilities in action_space.items():
        updated_weights = []

        for action, weight in action_probabilities.items():
            updated_weight = weight

            if weight > mean:
                updated_weight *= (1 + factor)
            else:
                updated_weight *= (1 - factor)

            # Ensure the updated weight is within the bounds [0, 10]
            updated_weight = max(0, min(updated_weight, 10))

            updated_weights.append(updated_weight)

        # # Normalize weights to sum up to 1
        # sum_weights = sum(updated_weights)
        # normalized_action_probabilities = {action: weight for action, weight in zip(action_probabilities.keys(), updated_weights)}

        # updated_action_space[gamestate] = normalized_action_probabilities
        updated_action_space[gamestate] = dict(zip(action_probabilities.keys(), updated_weights))

    return updated_action_space

In [11]:
def punish_action(action_space, mean, factor):
    updated_action_space = {}

    for gamestate, action_probabilities in action_space.items():
        updated_weights = []

        for action, weight in action_probabilities.items():
            updated_weight = weight

            if weight > mean:
                updated_weight -= (mean * factor)
            else:
                updated_weight += (mean * factor)

            # Ensure the updated weight is within the bounds [0, 10]
            updated_weight = max(0, min(updated_weight, 10))

            updated_weights.append(updated_weight)

        # # Normalize weights to sum up to 1
        # sum_weights = sum(updated_weights)
        # normalized_action_probabilities = {action: weight for action, weight in zip(action_probabilities.keys(), updated_weights)}

        # updated_action_space[gamestate] = normalized_action_probabilities
        updated_action_space[gamestate] = dict(zip(action_probabilities.keys(), updated_weights))

    return updated_action_space

In [12]:
def decrease_action(action_space, mean, factor):
    updated_action_space = {}

    for gamestate, action_probabilities in action_space.items():
        updated_weights = []

        for action, weight in action_probabilities.items():
            updated_weight = weight

            if weight > mean:
                updated_weight *= (1 - factor)
            else:
                updated_weight *= (1 + factor)

            # Ensure the updated weight is within the bounds [0, 10]
            updated_weight = max(0, min(updated_weight, 10))

            updated_weights.append(updated_weight)

        # # Normalize weights to sum up to 1
        # sum_weights = sum(updated_weights)
        # normalized_action_probabilities = {action: weight for action, weight in zip(action_probabilities.keys(), updated_weights)}

        # updated_action_space[gamestate] = normalized_action_probabilities
        updated_action_space[gamestate] = dict(zip(action_probabilities.keys(), updated_weights))

    return updated_action_space

# Reward Logic

In [13]:
class RewardSystem(Enum):
    Health = 1
    GoodAction = 2
    BadAction = 3
    Time = 4
    Explore = 5
    FoundDoor = 6

In [14]:
def updateReward(agent, metric):
    if metric == RewardSystem.Health:
        max_health = 100
        agent.Reward = agent.Reward - (max_health - agent.Health) * 0.5
    elif metric == RewardSystem.GoodAction:
        agent.Reward = agent.Reward + 2
    elif metric == RewardSystem.BadAction:
        agent.Reward = agent.Reward - 5
    elif metric == RewardSystem.Time:
        agent.Reward = agent.Reward - (agent.Counter * 0.25)
    elif metric == RewardSystem.Explore:
         agent.Reward = agent.Reward + 0.1
    elif metric == RewardSystem.FoundDoor:
        agent.Reward = agent.Reward + 20

# Agent Initialization

In [22]:
class Agent:
    def __init__(self, reward, health, position, action_space):
        self.Reward = reward
        self.Health = health
        self.max_Health = health
        self.Position = position
        self.Action_space = action_space
        self.Counter = 0
        
    def take_action(self, state):    
        actions = list(self.Action_space[state].keys())    
        probabilities = list(self.Action_space[state].values())    
        return random.choices(actions, weights=probabilities)[0]

    def updateReward(self, metric):
        if metric == RewardSystem.Health:
            self.Reward = self.Reward - (self.max_Health - self.Health) * 0.5
        elif metric == RewardSystem.GoodAction:
            self.Reward = self.Reward + 2
        elif metric == RewardSystem.BadAction:
            self.Reward = self.Reward - 5
        elif metric == RewardSystem.Time:
            # self.Reward = self.Reward - (self.Counter * 0.25)
            self.Reward = self.Reward - self.Counter
        elif metric == RewardSystem.Explore:
             self.Reward = self.Reward + 2e-5
        elif metric == RewardSystem.FoundDoor:
            self.Reward = self.Reward + 20

    def increment_counter(self):
        self.Counter += 1e-5

# Game Loop

In [25]:
from ReadWriteMemory import ReadWriteMemory

# Initialize metrics

# Set the desired duration for the loop in seconds
duration_in_seconds = 120 

#set the initial state
initial_state = Gamestate.Explore

#Memory Reader
rwm = ReadWriteMemory()
baseaddress = 0x64930000+0x00048184
addressOffset = [0x6B4, 0x4, 0x18, 0x6C, 0x28, 0x68, 0x12C]

# Create an agent instance with initial attributes and action space
initial_reward = 50
initial_health = 100
initial_position = (0, 0)
initial_action_space = generate_random_action_space(gamestates, actionList)


my_agent = Agent(initial_reward, initial_health, initial_position, initial_action_space)

#read data function
def read_data():
    f = open("savedList.py","r")
    data = f.read()
    f.close()
    return data

In [26]:
#Launch Game
os.startfile(r"C:\Users\rayan\OneDrive\Bureau\HackProject\CellsAI\Game\deadcells.exe")

# Wait for a few seconds to ensure the game has loaded
time.sleep(12)
   
#start Game
quickJump()
quickJump()

#Get game window
gameWindow = gw.getWindowsWithTitle('Dead Cells')[0]


# Record the start time
start_time = time.time()

#start reading health Data
process = rwm.get_process_by_name("deadcells.exe")
process.open()
healthpointer = process.get_pointer(baseaddress, offsets=addressOffset)

#record game state
state = initial_state

time.sleep(7)

max_health = 0
# Run the loop for the specified duration
while time.time() - start_time < duration_in_seconds:
    if my_agent.Health > 0 and my_agent.Health <= 100:
        #pick the state
        detection_list = read_data()
        if "enemy" in detection_list:
            if state != Gamestate.Enemy_fight:
                state = Gamestate.Enemy_fight
        else:
            state = Gamestate.Explore
        print('state:', state)
        
        #update time reward
        my_agent.increment_counter()
        my_agent.updateReward(RewardSystem.Time)
        print('counter:', my_agent.Counter)
        print('Reward:',my_agent.Reward)
        
        #get health value
        healthpointer = process.get_pointer(baseaddress, offsets=addressOffset)
        my_agent.Health = process.read(healthpointer)
        print('Health:',my_agent.Health)
        #update health
        if my_agent.Health < max_health:
            my_agent.updateReward(RewardSystem.Health)
        else:
            max_health = my_agent.Health
            my_agent.max_Health = max_health

        if state == Gamestate.Explore:
            my_agent.updateReward(RewardSystem.Explore)
            
        if gw.getActiveWindow() == gameWindow:
             
            # Pick a random action from the current state actionList
            action = my_agent.take_action(initial_state)
            # Call the randomly chosen action
            action()
                    
            if keyboard.is_pressed('p'):
                restart()
                print('Game restart')
                break
        if keyboard.is_pressed('esc'):
            print('Game loop is stopped')
            break

if my_agent.Reward < 50:
    my_agent.Action_space = reinforce_action(my_agent.Action_space, 5, my_agent.Reward/100)
else:
    my_agent.Action_space = decrease_action(my_agent.Action_space, 5, my_agent.Reward/100)

state: Gamestate.Explore
counter: 1e-07
Reward: 49.9999999
Health: 100
state: Gamestate.Explore
counter: 2e-07
Reward: 49.9999997
Health: 100
state: Gamestate.Explore
counter: 3e-07
Reward: 49.99999939999999
Health: 100
state: Gamestate.Explore
counter: 4e-07
Reward: 49.999998999999995
Health: 100
state: Gamestate.Explore
counter: 5e-07
Reward: 49.9999985
Health: 100
state: Gamestate.Explore
counter: 6e-07
Reward: 49.9999979
Health: 100
state: Gamestate.Explore
counter: 7e-07
Reward: 49.999997199999996
Health: 100
state: Gamestate.Explore
counter: 8e-07
Reward: 49.99999639999999
Health: 100
state: Gamestate.Explore
counter: 9e-07
Reward: 49.99999549999999
Health: 100
state: Gamestate.Explore
counter: 1e-06
Reward: 49.99999449999999
Health: 100
state: Gamestate.Explore
counter: 1.1e-06
Reward: 49.999993399999994
Health: 100
state: Gamestate.Explore
counter: 1.2e-06
Reward: 49.999992199999994
Health: 100
state: Gamestate.Explore
counter: 1.2999999999999998e-06
Reward: 49.99999089999999
H

In [ ]:
# # Create a dictionary of action sets with associated probabilities
# action_space = {
#     Gamestate.Enemy_fight: 
#     {
#         #actions and their probabilities
#         goRight : 0.2,
#         goLeft: 0.1,
#         jump: 0.15,
#         quickJump: 0.1,
#         crouch: 0.1,
#         long_crouch: 0.05,
#         doubleJump: 0.1,
#         quickRight: 0.05,
#         quickLeft: 0.05,
#         climb: 0.05,
#         roll: 0.05,
#         air_roll: 0.02,
#         weapon1: 0.03,
#         weapon2: 0.03,
#         attack_combo: 0.02,
#         collect: 0.02,
#         hardStomp: 0.03,
#         jumpDown: 0.02
#     },
#     Gamestate.Enemy_escape: 
#     {
#         #actions and their probabilities
#         goRight : 0.2,
#         goLeft: 0.1,
#         jump: 0.15,
#         quickJump: 0.1,
#         crouch: 0.1,
#         long_crouch: 0.05,
#         doubleJump: 0.1,
#         quickRight: 0.05,
#         quickLeft: 0.05,
#         climb: 0.05,
#         roll: 0.05,
#         air_roll: 0.02,
#         weapon1: 0.03,
#         weapon2: 0.03,
#         attack_combo: 0.02,
#         collect: 0.02,
#         hardStomp: 0.03,
#         jumpDown: 0.02
#     },
#     Gamestate.Explore: 
#     {
#         #actions and their probabilities
#         goRight : 0.2,
#         goLeft: 0.1,
#         jump: 0.15,
#         quickJump: 0.1,
#         crouch: 0.1,
#         long_crouch: 0.05,
#         doubleJump: 0.1,
#         quickRight: 0.05,
#         quickLeft: 0.05,
#         climb: 0.05,
#         roll: 0.05,
#         air_roll: 0.02,
#         weapon1: 0.03,
#         weapon2: 0.03,
#         attack_combo: 0.02,
#         collect: 0.02,
#         hardStomp: 0.03,
#         jumpDown: 0.02
#     },
#     Gamestate.Stuck: 
#     {
#         #actions and their probabilities
#         goRight : 0.2,
#         goLeft: 0.1,
#         jump: 0.15,
#         quickJump: 0.1,
#         crouch: 0.1,
#         long_crouch: 0.05,
#         doubleJump: 0.1,
#         quickRight: 0.05,
#         quickLeft: 0.05,
#         climb: 0.05,
#         roll: 0.05,
#         air_roll: 0.02,
#         weapon1: 0.03,
#         weapon2: 0.03,
#         attack_combo: 0.02,
#         collect: 0.02,
#         hardStomp: 0.03,
#         jumpDown: 0.02
#     },
#     Gamestate.Obstacle_Door: 
#     {
#         #actions and their probabilities
#         goRight : 0.2,
#         goLeft: 0.1,
#         jump: 0.15,
#         quickJump: 0.1,
#         crouch: 0.1,
#         long_crouch: 0.05,
#         doubleJump: 0.1,
#         quickRight: 0.05,
#         quickLeft: 0.05,
#         climb: 0.05,
#         roll: 0.05,
#         air_roll: 0.02,
#         weapon1: 0.03,
#         weapon2: 0.03,
#         attack_combo: 0.02,
#         collect: 0.02,
#         hardStomp: 0.03,
#         jumpDown: 0.02
#     },
#     Gamestate.Obstacle_Wall: 
#     {
#         #actions and their probabilities
#         goRight : 0.2,
#         goLeft: 0.1,
#         jump: 0.15,
#         quickJump: 0.1,
#         crouch: 0.1,
#         long_crouch: 0.05,
#         doubleJump: 0.1,
#         quickRight: 0.05,
#         quickLeft: 0.05,
#         climb: 0.05,
#         roll: 0.05,
#         air_roll: 0.02,
#         weapon1: 0.03,
#         weapon2: 0.03,
#         attack_combo: 0.02,
#         collect: 0.02,
#         hardStomp: 0.03,
#         jumpDown: 0.02
#     },
#     Gamestate.Obstacle_Floor: 
#     {
#         #actions and their probabilities
#         goRight : 0.2,
#         goLeft: 0.1,
#         jump: 0.15,
#         quickJump: 0.1,
#         crouch: 0.1,
#         long_crouch: 0.05,
#         doubleJump: 0.1,
#         quickRight: 0.05,
#         quickLeft: 0.05,
#         climb: 0.05,
#         roll: 0.05,
#         air_roll: 0.02,
#         weapon1: 0.03,
#         weapon2: 0.03,
#         attack_combo: 0.02,
#         collect: 0.02,
#         hardStomp: 0.03,
#         jumpDown: 0.02
#     }
# }

In [ ]:
# #each of the below methods simply sets which dictionary we are going to use for
# def enemyActions(): 
#     a_prob={
#         goRight : 0.2,
#         goLeft: 0.1,
#         jump: 0.15,
#         quickJump: 0.1,
#         crouch: 0.1,
#         long_crouch: 0.05,
#         doubleJump: 0.1,
#         quickRight: 0.05,
#         quickLeft: 0.05,
#         climb: 0.05,
#         roll: 0.05,
#         air_roll: 0.02,
#         weapon1: 0.03,
#         weapon2: 0.03,
#         attack_combo: 0.02,
#         collect: 0.02,
#         hardStomp: 0.03,
#         jumpDown: 0.02
#     }
#     return a_prob
    
# def exploreActions():
#     a_prob={
#         goRight : 0.2,
#         goLeft: 0.1,
#         jump: 0.15,
#         quickJump: 0.1,
#         crouch: 0.1,
#         long_crouch: 0.05,
#         doubleJump: 0.1,
#         quickRight: 0.05,
#         quickLeft: 0.05,
#         climb: 0.05,
#         roll: 0.05,
#         air_roll: 0.02,
#         weapon1: 0.03,
#         weapon2: 0.03,
#         attack_combo: 0.02,
#         collect: 0.02,
#         hardStomp: 0.03,
#         jumpDown: 0.02
#     }
#     return a_prob
    
# def stuckActions():
#     a_prob={
#         goRight : 0.2,
#         goLeft: 0.1,
#         jump: 0.15,
#         quickJump: 0.1,
#         crouch: 0.1,
#         long_crouch: 0.05,
#         doubleJump: 0.1,
#         quickRight: 0.05,
#         quickLeft: 0.05,
#         climb: 0.05,
#         roll: 0.05,
#         air_roll: 0.02,
#         weapon1: 0.03,
#         weapon2: 0.03,
#         attack_combo: 0.02,
#         collect: 0.02,
#         hardStomp: 0.03,
#         jumpDown: 0.02
#     }
#     return a_prob
    
# def doorActions():
#     a_prob={
#         goRight : 0.2,
#         goLeft: 0.1,
#         jump: 0.15,
#         quickJump: 0.1,
#         crouch: 0.1,
#         long_crouch: 0.05,
#         doubleJump: 0.1,
#         quickRight: 0.05,
#         quickLeft: 0.05,
#         climb: 0.05,
#         roll: 0.05,
#         air_roll: 0.02,
#         weapon1: 0.03,
#         weapon2: 0.03,
#         attack_combo: 0.02,
#         collect: 0.02,
#         hardStomp: 0.03,
#         jumpDown: 0.02
#     }
#     return a_prob
    
# def wallActions():
#     a_prob={
#         goRight : 0.2,
#         goLeft: 0.1,
#         jump: 0.15,
#         quickJump: 0.1,
#         crouch: 0.1,
#         long_crouch: 0.05,
#         doubleJump: 0.1,
#         quickRight: 0.05,
#         quickLeft: 0.05,
#         climb: 0.05,
#         roll: 0.05,
#         air_roll: 0.02,
#         weapon1: 0.03,
#         weapon2: 0.03,
#         attack_combo: 0.02,
#         collect: 0.02,
#         hardStomp: 0.03,
#         jumpDown: 0.02
#     }
#     return a_prob
    
# def floorActions():
#     a_prob={
#         goRight : 0.2,
#         goLeft: 0.1,
#         jump: 0.15,
#         quickJump: 0.1,
#         crouch: 0.1,
#         long_crouch: 0.05,
#         doubleJump: 0.1,
#         quickRight: 0.05,
#         quickLeft: 0.05,
#         climb: 0.05,
#         roll: 0.05,
#         air_roll: 0.02,
#         weapon1: 0.03,
#         weapon2: 0.03,
#         attack_combo: 0.02,
#         collect: 0.02,
#         hardStomp: 0.03,
#         jumpDown: 0.02
#     }
#     return a_prob

# # Define probabilities for each action (adjust these probabilities as needed)
# # Normalize probabilities to ensure they sum up to 1
# action_probabilities = doorActions()
# total_probability = sum(action_probabilities.values())
# action_probabilities = {action: prob / total_probability for action, prob in action_probabilities.items()}

# # Generate a dictionary with actions and their corresponding probabilities
# action_prob_dict = {actionList[i]: action_probabilities[actionList[i]] for i in range(len(actionList))}

# # Function to randomly choose an action based on probabilities
# def choose_action(action_probabilities):
#     rand_val = random.random()  # Generate a random number between 0 and 1
#     cumulative_prob = 0
    
#     for action, prob in action_probabilities.items():
#         cumulative_prob += prob
#         if rand_val <= cumulative_prob:
#             return action